# Instagram API - Automation
*Luís Eduardo Pompeu de Sousa Brasil Háteras*

In [31]:
# Show all output's on Jupyter Notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Authentication on Instagram

In [34]:
from instagram_private_api import Client, ClientCompatPatch

user_name = 'YOUR USERNAME'
password = 'YOUR PASSWORD'

api = Client(user_name, password)
results = api.feed_timeline()

## List of followers and following

In [32]:
from operator import itemgetter
import pandas as pd
import json
import requests

# How to get user's ID ? https://www.instagram.com/{username}/?__a=1
# or just type de username:
def get_userID(username):
    response = requests.get('https://www.instagram.com/{}/?__a=1'.format(username))
    dictionary = json.loads(response.text)
    return(dictionary['graphql']['user']['id'])

print(get_userID('edux21'))


# Following (while for pagination)
def following_users(userid_instagram):
    following = []
    combinacao=[]
    
    results = api.user_following(userid_instagram, rank_token=api.generate_uuid())
    following.extend(results.get('users', []))
    next_max_id = results.get('next_max_id')
    
    while next_max_id:
        results = api.user_following(userid_instagram, rank_token=api.generate_uuid(), max_id=next_max_id)
        following.extend(results.get('users', []))
        next_max_id = results.get('next_max_id')
        
    userid = [following[i]['pk'] for i in range(0,len(following))]
    full_names = [following[i]['full_name'] for i in range(0,len(following))]
    usernames = [following[i]['username'] for i in range(0,len(following))]
    profile_pic_url = [following[i]['profile_pic_url'] for i in range(0,len(following))]
    following_text = ['following' for i in range(0,len(following))]
    
    combinacao.extend([list(i) for i in zip(userid, full_names,
                                            usernames, profile_pic_url, following_text)])
    combinacao = sorted(combinacao, key=itemgetter(2), reverse=False)
    return(combinacao)
    
seguindo = following_users(get_userID('edux21'))
seguindo = pd.DataFrame(seguindo, columns = ['userID' , 'Full Name', 'username', 'Profile Picture', 'Type']) 
seguindo


# Followers (while for pagination)
def followers_users(userid_instagram):
    followers = []
    combinacao = []
    
    results = api.user_followers(userid_instagram, rank_token=api.generate_uuid())
    followers.extend(results.get('users', []))
    next_max_id = results.get('next_max_id')

    while next_max_id:
        results = api.user_followers(userid_instagram, rank_token=api.generate_uuid(), max_id=next_max_id)
        followers.extend(results.get('users', []))
        next_max_id = results.get('next_max_id')
        
    userid = [followers[i]['pk'] for i in range(0,len(followers))]
    full_names = [followers[i]['full_name'] for i in range(0,len(followers))]
    usernames = [followers[i]['username'] for i in range(0,len(followers))]
    profile_pic_url = [followers[i]['profile_pic_url'] for i in range(0,len(followers))]
    followers_text = ['follower' for i in range(0,len(followers))]
    
    combinacao.extend([list(i) for i in zip(userid, full_names,
                                            usernames, profile_pic_url, followers_text)])
    combinacao = sorted(combinacao, key=itemgetter(2), reverse=False)
    return(combinacao)

seguidores = followers_users(get_userID('edux21'))
seguidores = pd.DataFrame(seguidores, columns = ['userID' , 'Full Name', 'username', 'Profile Picture', 'Type']) 
seguidores

# Save in CSV file
seguindo.to_csv('following_usersInstagram.csv', mode='w', sep='\t', encoding='utf-16', index = None)
seguidores.to_csv('followers_usersInstagram.csv', mode='w', sep='\t', encoding='utf-16', index = None)




1624955639


,userID,Full Name,username,Profile Picture,Type
0,594016914,🙏,__sabrina.s,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
1,27523381240,Maria⭐️🌞🌙,_dudaquintela,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
2,2263530991,NEON PARTY,_neonpartyy,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
3,12873966052,"🌎 Fortaleza, Ceara. 🇧🇷",adnaalexander,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
4,986392897,Alixandra Mazini,alixmazini99,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
...,...,...,...,...,...
149,2110147578,Victor Linhares,victor_linharess,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
150,8479117510,⚕ Fisioterapeuta Ana Maria,visaoglobalfisio,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
151,1049836271,Vítor Arantes,vitimarantes,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
152,1579613509,Vitoria Goldini,vitoriagoldini_,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following


,userID,Full Name,username,Profile Picture,Type
0,308137197,Allison Villagra,____bizzle_____,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower
1,1459022478,❤ 방탄소년단 ❤,_bangtan_boys_forever_,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower
2,376075950,Bruna Leite ✨,_brunaleiteg,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower
3,1320974570,Daniel Joseph Cardona,_daniel125_,https://instagram.fktp1-1.fna.fbcdn.net/v/t51....,follower
4,1620694894,,_desativaaadoo,https://instagram.fplm3-1.fna.fbcdn.net/v/t51....,follower
...,...,...,...,...,...
275,464528789,Stephanie Monroe,xostephanierenee,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower
276,1615751054,yan jorge,yan_jorge_,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower
277,1619976343,,zoarizar,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower
278,1609687008,Zoeiras do Brasil,zoeirasdobrasil,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower


## Mutual friends, Following alone, Follower alone

In [33]:
# Following + Followers
df = pd.concat([seguidores,seguindo], axis=0).reset_index()
total_following_followers = len(df)

# Mutual friends - following and follower
mutual_friends = df[df.duplicated(subset='username')].reset_index()
mutual_friends.drop(columns=['level_0', 'index', 'Type'], inplace=True)
mutual_friends

# People who follows you and you don't follow back or 
# People who you follow and they don't follow you back
df = df.drop_duplicates(subset=['username'], keep=False)

# Following people but who don't follow you back
following_alone = df[df['Type']=='following'].reset_index()
following_alone.drop(columns=['level_0', 'index'], inplace=True)
following_alone

# Followers who you don't follow them back
followers_alone = df[df['Type']=='follower'].reset_index()
followers_alone.drop(columns=['level_0', 'index'], inplace=True)
followers_alone


# Save in CSV file
mutual_friends.to_csv('mutualFriendsInstagram.csv', mode='w', sep='\t', encoding='utf-16', index = None)
following_alone.to_csv('following_alone.csv', mode='w', sep='\t', encoding='utf-16', index = None)
followers_alone.to_csv('followers_alone.csv', mode='w', sep='\t', encoding='utf-16', index = None)


print('Mutual friends total (%): ' + str(100*2*len(mutual_friends)/total_following_followers))
print('Following people who dont follow you back (%): ' + str(100*len(following_alone)/total_following_followers))
print('Followers who you dont follow them back (%): ' + str(100*len(followers_alone)/total_following_followers))

,userID,Full Name,username,Profile Picture
0,27523381240,Maria⭐️🌞🌙,_dudaquintela,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
1,1387166444,Ana Gabriela Bezerra,anagabib,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
2,460278242,Ana,anaipirangag,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
3,4729506000,Andresa Oliveira,andresa_oliver01,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
4,6434399329,Artur Nogueira Brito,arturnogueirab,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
...,...,...,...,...
76,1976772442,Antonia Ariane,toniaariane,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
77,2110147578,Victor Linhares,victor_linharess,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
78,8479117510,⚕ Fisioterapeuta Ana Maria,visaoglobalfisio,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
79,1049836271,Vítor Arantes,vitimarantes,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....


,userID,Full Name,username,Profile Picture,Type
0,594016914,🙏,__sabrina.s,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
1,2263530991,NEON PARTY,_neonpartyy,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
2,12873966052,"🌎 Fortaleza, Ceara. 🇧🇷",adnaalexander,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
3,986392897,Alixandra Mazini,alixmazini99,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
4,909177565,Mariana Barbosa,amarianabarbosa_,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
...,...,...,...,...,...
68,4323918335,Two Folks,twofolkss,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
69,1508590915,Universidade Federal do Ceará,ufcinforma,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
70,14555646520,UFC ordinária,ufcordinaria,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
71,55273796,Úrsula,ursulaamaral,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following


,userID,Full Name,username,Profile Picture,Type
0,308137197,Allison Villagra,____bizzle_____,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower
1,1459022478,❤ 방탄소년단 ❤,_bangtan_boys_forever_,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower
2,376075950,Bruna Leite ✨,_brunaleiteg,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower
3,1320974570,Daniel Joseph Cardona,_daniel125_,https://instagram.fktp1-1.fna.fbcdn.net/v/t51....,follower
4,1620694894,,_desativaaadoo,https://instagram.fplm3-1.fna.fbcdn.net/v/t51....,follower
...,...,...,...,...,...
194,1618757817,lyss,wiccaniando,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower
195,464528789,Stephanie Monroe,xostephanierenee,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower
196,1619976343,,zoarizar,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower
197,1609687008,Zoeiras do Brasil,zoeirasdobrasil,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,follower


Mutual friends total (%): 37.327188940092164
Following people who dont follow you back (%): 16.820276497695854
Followers who you dont follow them back (%): 45.85253456221198


## Photos - Date of publication, location, number of likes, comments, URL

In [357]:
import pandas as pd
from collections import Counter
import datetime

# Creating dataframe
df = pd.DataFrame()


# Extract photos information
def photo_information(username_insta):
    
    # Initialize the lists
    likes=[]
    comments_count=[]
    url=[]
    teste=[]
    data_foto=[]
    latitudelista = []
    longitudelista = []
    locationlista = []
    caption_photo=[]
    curtidores_username=[]
    curtidores_fullname=[]
    username_lista=[]

    # Extract all photos information (while for pagination)
    request = api.username_feed(username_insta)
    teste.extend(request.get('items'))
    next_max_id = request.get('next_max_id')
    while next_max_id:
        request = api.username_feed(username_insta, max_id=next_max_id)
        next_max_id = request.get('next_max_id')
        teste.extend(request.get('items'))
    

    # Número de likes em todas as fotos
    for i in range(0,len(teste)):
        # username
        username_lista.append(username_insta)

        # Date
        if 'taken_at' in teste[i]:
            data_foto.append(datetime.datetime.utcfromtimestamp(teste[i]['taken_at']).strftime('%Y-%m-%d %H:%M:%S'))
        else:
            data_foto.append('-')

        # Caption text photo
        if ('caption' in teste[i]) and (not teste[i]['caption'] is None):
            titulo_foto=str(teste[i]['caption']['text'])
            caption_photo.append(titulo_foto)
        else:
            caption_photo.append('-')

        # Number of likes
        if 'like_count' in teste[i]:
            likes.append(teste[i]['like_count'])
        else:
            likes.append('-')

        # Comments count
        if 'comment_count' in teste[i]:
            comments_count.append(teste[i]['comment_count'])
        else:
            comments_count.append('-')

        # Latitude, longitude, location
        if 'lat' in teste[i]:
            latitudelista.append(teste[i]['lat'])
        else:
            latitudelista.append('-')
            
        if 'lng' in teste[i]:    
            longitudelista.append(teste[i]['lng'])
        else:
            longitudelista.append('-')
            
            
        if 'location' in teste[i]:   
            locationlista.append(teste[i]['location']['city'])
        else:
            locationlista.append('-')

        # URL
        if 'carousel_media' not in teste[i]:
            url.append(teste[i]['image_versions2']['candidates'][0]['url'])
        else:
            url.append(teste[i]['carousel_media'][0]['image_versions2']['candidates'][0]['url'])
    
    combinacao21=[]
    combinacao21.extend([list(i) for i in zip(data_foto, username_lista,
                                        likes, comments_count, caption_photo, locationlista, latitudelista, longitudelista, url)])
    combinacao21 = sorted(combinacao21, key=itemgetter(1), reverse=False)
    return(combinacao21)

    
vamosla = photo_information('eduardocandido6')  
vamosla = pd.DataFrame(vamosla, columns = ['Data' , 'Username', 'Likes', 'Comments', 'Title Photo', 'Location', 'Latitude', 'Longitude', 'URL']) 
vamosla['Data'] = pd.to_datetime(vamosla['Data'])
# vamosla['year'] = vamosla['Data'].dt.year
vamosla

# Save in CSV file
vamosla.to_csv('photosInformation.csv', mode='w', sep='\t', encoding='utf-16', index = None)

# --- Top and Worst Photo

# Worst Photo
min_likes = vamosla[vamosla['Likes']==vamosla['Likes'].min()]
min_likes

# Best Photo
max_likes = vamosla[vamosla['Likes']==vamosla['Likes'].max()]
max_likes

# Average number of likes
vamosla['Likes'].mean()

# Median number of likes
vamosla['Likes'].median()


,Data,Username,Likes,Comments,Title Photo,Location,Latitude,Longitude,URL
0,2020-04-03 20:45:04,eduardocandido6,58,6,"E lá se vão 3 anos dessa foto, mas merece ser ...","São Paulo (São Paulo, Brazil)",-23.5291,-46.5311,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
1,2020-03-13 01:18:27,eduardocandido6,62,2,"#Tbt dos tempos de tranquilidade, praia e sol",,-2.7956,-40.5148,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
2,2020-02-22 19:29:06,eduardocandido6,40,0,Carnaval 2020,-,-,-,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
3,2020-02-09 16:38:53,eduardocandido6,84,11,⛵☀️,,-22.948,-43.1633,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
4,2020-02-03 10:43:14,eduardocandido6,69,4,Aniversário dos galados nessa cidade marcante ...,,-22.908,-43.1975,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
...,...,...,...,...,...,...,...,...,...
61,2015-10-17 23:56:51,eduardocandido6,20,2,Poderia estar na NBA tranquilamente.,-,-,-,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
62,2015-10-05 17:12:40,eduardocandido6,21,2,Em fortaleza com mamis!,-,-,-,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
63,2015-10-02 15:27:18,eduardocandido6,12,0,-,-,-,-,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
64,2015-09-29 04:54:24,eduardocandido6,13,0,1° churrasco da educariun!,-,-,-,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....


,Data,Username,Likes,Comments,Title Photo,Location,Latitude,Longitude,URL
63,2015-10-02 15:27:18,eduardocandido6,12,0,-,-,-,-,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....


,Data,Username,Likes,Comments,Title Photo,Location,Latitude,Longitude,URL
15,2018-08-15 13:24:13,eduardocandido6,115,5,Últimas fotos aproveitando esse Instituto #bu...,-,-,-,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....


52.89393939393939

53.0

## Download all photos

In [293]:
import os, urllib.request
import random

def download_all_images(username_insta, folder):
    os.mkdir(folder)
    listas = photo_information(username_insta)
    for i in range(0,len(listas)):
        endereco = folder + '//' + username_insta + str(listas[i][-3]) + '&' +  str(listas[i][-2]) +'_'+str(random.randint(1,1001))+'.jpg'
        urllib.request.urlretrieve(listas[i][-1], endereco)  
        print(listas[i][-1])

download_all_images('victor_linharess', 'Fotos_victor')

https://instagram.fnat1-1.fna.fbcdn.net/v/t51.2885-15/e35/47690932_1984395141676693_5796752888539015947_n.jpg?_nc_ht=instagram.fnat1-1.fna.fbcdn.net&_nc_cat=101&_nc_ohc=TqbmmjoisGMAX-nCfYn&se=7&oh=ec39927edca492a4e6fb2118ecf9c9f0&oe=5ED43ABC&ig_cache_key=MTk0ODM0OTM0ODExMjY4Nzk0OA%3D%3D.2
https://instagram.fnat1-1.fna.fbcdn.net/v/t51.2885-15/e35/34417809_852645114921171_4828483035382939648_n.jpg?_nc_ht=instagram.fnat1-1.fna.fbcdn.net&_nc_cat=103&_nc_ohc=3GETw9B5thcAX-HIUP5&se=7&oh=f1ed56ba300dec0ff84d4dfe15fff621&oe=5ED0C0DC&ig_cache_key=MTc5ODkzMDU2ODQwMDYzMjA3Ng%3D%3D.2
https://instagram.fnat1-1.fna.fbcdn.net/v/t51.2885-15/e35/31738638_603320116671411_2524041577077145600_n.jpg?_nc_ht=instagram.fnat1-1.fna.fbcdn.net&_nc_cat=103&_nc_ohc=_EMBXccRJz8AX-vuUVi&se=8&oh=ddfc133e67533557883587d14ac3011e&oe=5ED131BA&ig_cache_key=MTc4MzU3Njk5MDM5NDE0NDg3Mg%3D%3D.2
https://instagram.fnat1-1.fna.fbcdn.net/v/t51.2885-15/e35/31435449_1556426694456605_1623336211542179840_n.jpg?_nc_ht=instagram.fnat1

## People who most like any user photos

In [359]:
# Media-id: https://api.instagram.com/oembed/?url=https://www.instagram.com/p/B_SUEKaDZhK/
# Extract "media_id" from all photos
def mediaid(username_insta):
    mediaid=[]
    request = api.username_feed(username_insta)
    mediaid.extend([request['items'][i]['pk'] for i in range(0, len(request['items']))])
    next_max_id = request.get('next_max_id')
    while next_max_id:
        request = api.username_feed(username_insta, max_id=next_max_id)
        mediaid.extend([request['items'][i]['pk'] for i in range(0, len(request['items']))])
        next_max_id = request.get('next_max_id')
    return(mediaid)

listacomids = mediaid('eduardocandido6')
len(listacomids)

# Likers from all photos (1000 likes limitation per photo)
def likers(username_insta):
    curtidores_id=[]
    curtidores_username=[]
    curtidores_fullname=[]
    curtidores_profile_pic=[]
    combinacao=[]
    lista_image_id = mediaid(username_insta)
    # Change "len(lista_image_id)" to number of photos you want to extract - default = all
    for i in range(0, len(lista_image_id)):
        for curtidas in api.media_likers(lista_image_id[i]).get('users'):
            curtidores_username.append(curtidas['username'])
            curtidores_fullname.append(curtidas['full_name'])
            curtidores_id.append(curtidas['pk'])
            curtidores_profile_pic.append(curtidas['profile_pic_url'])
    
    combinacao.extend([list(i) for i in zip(curtidores_id, curtidores_fullname,
                                        curtidores_username, curtidores_profile_pic)])
    combinacao = sorted(combinacao, key=itemgetter(2), reverse=False)
    return(combinacao)
    
# Likers - complete list
curtidores = likers('eduardocandido6')
curtidores = pd.DataFrame(curtidores, columns = ['UserID' , 'Name', 'Username', 'URL Person']) 
curtidores

# Counting likes by username or name
curtidores_username_filtrado=Counter(curtidores['Username']).most_common()
curtidores_fullname_filtrado=Counter(curtidores['Name']).most_common()
curtidores_username_filtrado = pd.DataFrame(curtidores_username_filtrado, columns = ['Username' , 'Qtd'])
curtidores_username_filtrado

# Save in CSV file
curtidores_username_filtrado.to_csv('likers.csv', mode='w', sep='\t', encoding='utf-16', index = None)



66

,UserID,Name,Username,URL Person
0,371449261,,__mayaracarvalh0,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
1,371449261,,__mayaracarvalh0,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
2,371449261,,__mayaracarvalh0,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
3,274742249,,__miferreira,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
4,274742249,,__miferreira,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
...,...,...,...,...
3315,29893198,Yvan Tourinho,yvantourinho,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
3316,29893198,Yvan Tourinho,yvantourinho,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
3317,29893198,Yvan Tourinho,yvantourinho,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....
3318,29893198,Yvan Tourinho,yvantourinho,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....


## People who most comment any user photos

In [360]:
# Extract the first 500 user's that commented on a photo
def comentaristas(username_insta):
    lista_media_id = mediaid(username_insta)
    comentaristas=[]
    # Change "len(lista_image_id)" to number of photos you want to extract - default = all
    for i in range(0,len(lista_media_id)):
        comentarios = api.media_n_comments(lista_media_id[i], n=500)
        for n_com in range(0,len(comentarios)):
            comentaristas.append(comentarios[n_com]['user']['username'])
    return(comentaristas)
        
pessoas_comentam = comentaristas('eduardocandido6')
principais_comentaristas=Counter(pessoas_comentam).most_common()
principais_comentaristas = pd.DataFrame(principais_comentaristas, columns = ['Username' , 'Qtd'])
principais_comentaristas

# Save in CSV file
principais_comentaristas.to_csv('commenters.csv', mode='w', sep='\t', encoding='utf-16', index = None)


,Username,Qtd
0,eduardocandido6,34
1,renatacborgess,26
2,taniacandidoborges,24
3,brunocborgess,16
4,pedrosiau,12
...,...,...
77,talizefaco,1
78,lucascawai,1
79,guizaa1,1
80,asd1233y,1


## Like all the photos of an user

In [24]:
# Curtir todas as fotos da pessoa
listacomids = mediaid('eduardocandido6')
for i in range(0,len(listacomids)):
    api.post_like(listacomids[i], module_name='feed_timeline')

ClientError: Bad Request: Not authorized to view user

## Other useful functions

In [43]:
import requests
import json

# Get username ID
get_userID('quebrandootabu')

# Convert media_id to URL link:
api.media_permalink(2292983407267649610)['permalink']

# Get media_id using photo URL
def URLtoMediaID(url):
    texto = requests.get('https://api.instagram.com/oembed/?url={}'.format(url)).text
    formatado = json.loads(texto)
    string = formatado['media_id']
    return(string[:string.find('_')])
URLtoMediaID('https://www.instagram.com/p/4a03XqpveL/')

# Post a comment - (media_id, text)
#api.post_comment(1016357169326913419, 'Teste :)')

# Check userid is following a tag
api.tags_user_following(196211285)

# User detail info - experimental
#api.user_detail_info(1624955639)

# Check userID feed
#api.user_feed(7578321540)

# Follow someone (userid)
api.friendships_create(7578321540)

# Unfollow
#friendships_destroy(userid)

# Friendship status
api.friendships_show(get_userID('luanaravena1'))

# Requests of friendship pending
api.friendships_pending()['users']

# Get user feed and story information
api.user_story_feed(get_userID('manueladavila'))

# Get user stories
api.user_story_feed(get_userID('tiaoviana'))['reel']

# Get comments likers
#api.comment_likers(17893984120500475)

# Get comments replies
#api.comment_replies(URLtoMediaID('https://www.instagram.com/p/B_kndbCpkvx/'), 17893984120500475)

# More...
# https://instagram-private-api.readthedocs.io/en/latest/api.html

'7578321540'

'https://www.instagram.com/p/B_SUEKaDZhK/?igshid=18aejv0eoqc9x'

'1016357169326913419'

{'tags': [], 'status': 'ok'}

ClientThrottledError: Please wait a few minutes before you try again.

## Using WebAPI:

In [58]:
# WebAPI

import hashlib
import string
import random
import pprint
from instagram_web_api import Client, ClientCompatPatch, ClientError, ClientLoginError

class MyClient(Client):
    @staticmethod
    def _extract_rhx_gis(html):
        options = string.ascii_lowercase + string.digits
        text = ''.join([random.choice(options) for _ in range(8)])
        return hashlib.md5(text.encode())

# Without any authentication
web_api = MyClient(auto_patch=True, drop_incompat_keys=False)

# Get location feed (location_id) or location_story_feed(location_id)
web_api.location_feed(273471170716)


{'data': {'location': {'id': '1924834',
   'name': 'San Francisco Honda',
   'has_public_page': True,
   'lat': 37.7749097,
   'lng': -122.4193514,
   'slug': 'san-francisco-honda',
   'blurb': "San Francisco Honda, one of America's original 50 Honda dealerships, is a family-owned business dedicated to the highest standards of customer and community service.\nwww.sfhonda.com",
   'website': 'https://www.sfhonda.com/',
   'phone': '(415) 687-0174',
   'primary_alias_on_fb': 'SFHonda',
   'address_json': '{"street_address": "1595 Van Ness Ave", "zip_code": "94109", "city_name": "San Francisco, California", "region_name": "", "country_code": "US", "exact_city_match": false, "exact_region_match": false, "exact_country_match": false}',
   'profile_pic_url': 'https://instagram.fnat1-1.fna.fbcdn.net/v/t51.2885-15/e35/c0.135.1080.1080a/s150x150/47585891_337877943433037_1661686589533243031_n.jpg?_nc_ht=instagram.fnat1-1.fna.fbcdn.net&_nc_cat=105&_nc_ohc=iUDjfZrYwbwAX_lFiEq&oh=15240c5b485a5133ae

AttributeError: 'MyClient' object has no attribute 'comment_replies'

## How to be the first comment on a popular instagram's page

In [27]:
from instagram_private_api import Client, ClientCompatPatch
import time
from covid import Covid

# Extract covid-19 data
covid = Covid(source="worldometers")

userid_insta = 4544395091
feed_qt = api.user_feed(userid_insta)

# Last publication
last_publication = feed_qt['items'][0]['pk']

while True:
    new_publication = api.user_feed(userid_insta)['items'][0]['pk']
    mortes = covid.get_status_by_country_name("brazil")['deaths']
    if new_publication!=last_publication:
        frase_publicacao = 'Número de mortes no Brasil COVID-19: {}'.format(mortes)
        api.post_comment(new_publication, frase_publicacao)
        print(api.media_permalink(new_publication)['permalink'])
        last_publication=new_publication
    time.sleep(10)

https://www.instagram.com/p/B_kl7YZpHAD/?igshid=edixea70y331
https://www.instagram.com/p/B_kndbCpkvx/?igshid=w59rkuq1ywyf
https://www.instagram.com/p/B_kpag2pXxC/?igshid=122ac4mnvg9ps
https://www.instagram.com/p/B_krxrhpQ8S/?igshid=1ek09sfbmimjq
https://www.instagram.com/p/B_lADElpu9q/?igshid=1hstnbpnin56e
https://www.instagram.com/p/B_lAI-aJF8n/?igshid=bb2eemyc4nyj


KeyboardInterrupt: 

## Multiple pages:

In [92]:
from instagram_private_api import Client, ClientCompatPatch
import time

# Initialize dictionaries
last_publication = {}
new_publication = {}

#opovoonline

# Usernames
usernames = ['quebrandootabu', 'jornaloglobo', 'conexaopoliticabrasil', 'brasil.de.direita', 
             'mblivre', 'joicehasselmannoficial', 'cristiano', 'beyonce', 'senadorhumberto', 'portalg1', 'bolsonarosp',
            'jairmessiasbolsonaro', 'ptbrasil', 'jornalistaslivres']

# Get the last publication
for i in range(0, len(usernames)):
    last_publication[i] = api.username_feed(usernames[i])['items'][0]['pk']
    
while True:
    
    # Text to post
    frase_publicacao = 'Bom dia'
    
    
    for i in range(0, len(usernames)):
        # Get new publication
        new_publication[i] = api.username_feed(usernames[i])['items'][0]['pk']
        
        # If there is a new publication then post a comment
        if new_publication[i] != last_publication[i]:
            api.post_comment(new_publication[i], frase_publicacao)
            print(api.media_permalink(new_publication[i])['permalink'])
            last_publication[i]=new_publication[i]

# Wait 10 seconds
time.sleep(10)

KeyboardInterrupt: 

## Follow multiple users

In [78]:
seguindo = following_users(get_userID('whinderssonnunes'))
seguindo = pd.DataFrame(seguindo, columns = ['userID' , 'Full Name', 'username', 'Profile Picture', 'Type']) 
seguindo

for i in seguindo['userID']:
    api.friendships_create(i)

,userID,Full Name,username,Profile Picture,Type
0,2086200625,Dagoberto Pelentier,11dagoberto,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
1,1803786393,Rebecca King,3gungal,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
2,254581750,88 Tactical of Omaha,88tactical,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
3,673955424,PMESP,_pmesp,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
4,1215266817,Sara Winter,_sarawinter,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
...,...,...,...,...,...
625,2257447881,Yair Netan,yairnetan_,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
626,4017539135,Yossi Shelley,yossishelley,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
627,2467734432,Ysani Kalapalo,ysanikalapalo,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following
628,2235904150,Yves Daberkow de Sousa,ysousa777,https://instagram.fnat1-1.fna.fbcdn.net/v/t51....,following


ClientError: Bad Request: feedback_required

## Post a comment in all photos of a person

In [23]:
# Curtir todas as fotos da pessoa
listacomids = mediaid(username)
for i in range(4,len(listacomids)):
    frase_publicacao = 'Bom dia !' 
    api.post_comment(listacomids[i], frase_publicacao)
    print(listacomids[i])

ClientError: Bad Request: feedback_required